# Getting timestamps on long audio

> "With wav2vec2 and HuggingFace transformers"

- tok: false
- branch: master
- comments: true
- categories: [long audio, wav2vec2, huggingface, timestamps]


First, an audio sample. Using [this video](https://www.youtube.com/watch?v=Kw5jkyLGFGc) from youtube. Youtube says it's 11 minutes, 51 seconds, so that should be enough to check that striding works.

In [ ]:
!pip install youtube-dl

In [2]:
!youtube-dl -x --audio-format best -o '%(id)s.%(ext)s' https://www.youtube.com/watch?v=Kw5jkyLGFGc

[youtube] Kw5jkyLGFGc: Downloading webpage
[youtube] Kw5jkyLGFGc: Downloading MPD manifest
[download] Destination: Kw5jkyLGFGc.m4a
[download] 100% of 10.98MiB in 02:25
[ffmpeg] Correcting container in "Kw5jkyLGFGc.m4a"
[ffmpeg] Post-process file Kw5jkyLGFGc.m4a exists, skipping


In [3]:
!ffmpeg -i Kw5jkyLGFGc.m4a -acodec pcm_s16le -ac 1 -ar 16000 Kw5jkyLGFGc.wav

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

Here starts the actual ASR stuff.

In [4]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 4.1 MB/s 
     |████████████████████████████████| 67 kB 5.6 MB/s 
     |████████████████████████████████| 6.5 MB 41.7 MB/s 
     |████████████████████████████████| 895 kB 45.7 MB/s 
     |████████████████████████████████| 596 kB 74.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
_SWE_MODEL = "KBLab/wav2vec2-large-voxrex-swedish"

In [6]:
from transformers import pipeline

In [7]:
pipe = pipeline(model=_SWE_MODEL)

Downloading:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:357: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/211 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/421 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/212 [00:00<?, ?B/s]

For working with strides, there's information in a [blog post](https://huggingface.co/blog/asr-chunking).

There isn't much information on getting timestamps from a pipeline, but the detail is in the [pull request](https://github.com/huggingface/transformers/pull/15792).

In [8]:
output = pipe("/content/Kw5jkyLGFGc.wav", chunk_length_s=10)

In [9]:
output = pipe("/content/Kw5jkyLGFGc.wav", chunk_length_s=10, return_timestamps="word")

In [10]:
import json
with open("/content/Kw5jkyLGFGc.json", "w") as f:
    json.dump(output, f)